In [ ]:
import re
import numpy as np
from scripts.scrape import get_data, get_test_data
import itertools


# Day 15

In [ ]:
data = get_data(15)
test = get_test_data(15)
data

In [ ]:
data = [int(d) for d in data[0].split(',')]

In [ ]:
seq = data.copy()

last = {}
for i in range(len(seq)):
    last[seq[i]] = [i, i]
for i in range(len(seq), 2020):
    lookup = seq[i-1]
    val = last[lookup][1] - last[lookup][0]
    seq.append(val)
    if val in last.keys():
        last[val] = [last[val][1], i]
    else: 
        last[val] = [i, i]        
seq[-1]

In [ ]:
# Part 2

seq = data.copy()

last = {}
for i in range(len(seq)):
    last[seq[i]] = [i, i]
for i in range(len(seq), 30000000):
    lookup = seq[i-1]
    val = last[lookup][1] - last[lookup][0]
    seq.append(val)
    if val in last.keys():
        last[val] = [last[val][1], i]
    else: 
        last[val] = [i, i]        

seq[-1]

# Day 16

In [ ]:
data = get_data(16,'\n\n')
test = get_test_data(16)
data

In [ ]:
data = [d.split('\n') for d in data]

In [ ]:
validation = data[0]
my_ticket = data[1][1:]
nearby_tickets = data[2][1:]

In [ ]:
def validate(number, validation):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
#     print(validation_ranges)
    valid = [(number >= int(v[0])) & (number <= int(v[1])) for v in validation_ranges]
#     print(valid)
    if np.any(valid):
        return(0)
    else: return(number)
    

In [ ]:
np.sum([validate(int(n), validation) for t in nearby_tickets for n in t.split(',')])

### Part 2

In [ ]:
def validate(number, validation):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
    valid = [(number >= int(v[0])) & (number <= int(v[1])) for v in validation_ranges]
    if np.any(valid):
        return(True)
    else: return(False)
    

In [ ]:
mask = [np.all([validate(int(n), validation) for n in t.split(',')]) for t in nearby_tickets]
nearby_tickets = np.array([[int(n) for n in t.split(',')] for t in nearby_tickets])
nearby_tickets = nearby_tickets[mask]
nearby_tickets

In [ ]:
def guess(number):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
    valid = [(number >= int(v[0])) & (number <= int(v[1])) for v in validation_ranges]
    return(np.array(valid)[np.arange(1, 40, 2)] + np.array(valid)[np.arange(0, 39, 2)])
guess(337)

In [ ]:
truth_values = np.apply_along_axis(guess, 1, nearby_tickets)
logic_matrix = np.array([np.apply_along_axis(np.all, 0, truth_values[:,i,:]) for i in range(20)])
assignment = {}
for i in range(20):
    row = np.where(logic_matrix.sum(1) == 1)[0][0]
    column = np.where(logic_matrix[row, :])[0][0]
    assignment[column] = row
    logic_matrix[:, column] = 0
np.prod([int(my_ticket[0].split(',')[v]) for v, k in assignment.items() if k in range(6)])    

In [ ]:
def check_ticket(ticket):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
    valid = [(int(validation_ranges[2*assignment[idx]][0]) <= item <=  int(validation_ranges[2*assignment[idx]][1])) + \
        (int(validation_ranges[2*assignment[idx] + 1][0]) <= item <=  int(validation_ranges[2*assignment[idx]+ 1][1])) for idx, item in enumerate(ticket)]
    return(np.all(valid))
np.all([check_ticket(t) for t in nearby_tickets])

# Day 17

In [ ]:
data = get_data(17)
test = get_test_data(17)
data

In [ ]:
cube = np.zeros([22,22,15])
starter = np.array([list(re.sub('\.', '0', re.sub('#', '1', d))) for d in data])
cube[8:16, 8:16, 7] = starter

In [ ]:
def grow(cube, count = 1):
    new_cube = np.zeros([22,22,15])
    for idx in itertools.product(np.arange(1,22), np.arange(1,22), np.arange(1,15)):
        new_cube[idx] = cell(cube[(idx[0]-1):(idx[0]+2), 
                                  (idx[1]-1):(idx[1]+2), 
                                  (idx[2]-1):(idx[2]+2)])
    if count == 1:    
        return(new_cube)
    else:
        return(grow(new_cube, count-1))
    
def cell(cube):
    center = cube[1,1,1]
    if (bool(center) & (cube.sum() in [3,4])) | (~bool(center) & (cube.sum() == 3)):
        return(1)
    else: 
        return(0)
        

In [ ]:
np.sum(grow(cube, 6))

### Part 2

In [ ]:
cube = np.zeros([22,22,15,15])
starter = np.array([list(re.sub('\.', '0', re.sub('#', '1', d))) for d in data])
cube[8:16, 8:16, 7, 7] = starter

def grow(cube, count = 1):
    new_cube = np.zeros([22,22,15, 15])
    for idx in itertools.product(np.arange(1,22), np.arange(1,22), np.arange(1,15), np.arange(1,15)):
        new_cube[idx] = cell(cube[(idx[0]-1):(idx[0]+2), \
                                  (idx[1]-1):(idx[1]+2), \
                                  (idx[2]-1):(idx[2]+2), \
                                  (idx[3]-1):(idx[3]+2)])
    if count == 1:    
        return(new_cube)
    else:
        return(grow(new_cube, count-1))
    
def cell(cube):
    center = cube[1,1,1,1]
    if (bool(center) & (cube.sum() in [3,4])) | (~bool(center) & (cube.sum() == 3)):
        return(1)
    else: 
        return(0)
        

In [ ]:
grow(cube, 6).sum()

# Day 18

In [ ]:
data = get_data(18)
test = get_test_data(18)
data

In [ ]:
def eval_expr(expr):
    new_expr = re.sub("([0-9]+)", "\\1)", expr)
    open_parens = np.sum([(l==")") - (l=="(") for l in list(new_expr)])
    return eval('(' * open_parens + new_expr)

In [ ]:
def parse(expr):
    innermost = re.findall('\([^()]*\)', expr)
    replacements = [(i, str(eval_expr(i[1:-1]))) for i in innermost]
    for item in replacements:
        expr = expr.replace(*item)
    if '(' in expr:
        return(parse(expr))
    else:
        return(eval_expr(expr))

            

In [ ]:
np.sum([parse(d) for d in data])

### Part 2

In [ ]:
# Part 2

class num(int):
    def __add__(self, a):
        return num(self.__int__() * a)
    def __mul__(self, a):
        return num(self.__int__() + a)
        


In [ ]:
np.sum([eval(re.sub('([0-9]+)', 'num(\\1)', d.translate(str.maketrans('*+','+*')))) for d in data])

# Day 19

In [ ]:
data = get_data(19, '\n\n')
test = get_test_data(19)

In [ ]:
data_rules = data[0].split('\n')
data_values = data[1].split('\n')


In [ ]:
def parse_data(data):
    rules = {}
    for item in data:
        key, value = re.split(':', item)
        rules[int(key)] = value.strip(' ')
    return(rules)
rules = parse_data(data_rules)

In [ ]:
num_extract = re.compile('(?<![0-9])([0-9]+)(?![0-9])')
while(re.search("[0-9]", rules[0])):
    for item in num_extract.findall(rules[0]):
        replacement = rules[int(item)]
        if '|' in replacement:
            replacement = '('+replacement+')'
        rules[0] = re.sub('(?<![0-9])'+item+'(?![0-9])', replacement, rules[0])

rules[0] = re.sub(' ', '', rules[0])
rules[0] = re.sub('"', '', rules[0])
rules[0] = '^' + rules[0] + '$'

In [ ]:
validate = re.compile(rules[0])

In [ ]:
np.sum([True if validate.search(d) else False for d in data_values ])

### Part 2

In [ ]:
rules = parse_data(data_rules)
num_extract = re.compile('(?<![0-9])([0-9]+)(?![0-9])')
while(re.search("[0-9]", rules[42])):
    for item in num_extract.findall(rules[42]):
        replacement = rules[int(item)]
        if '|' in replacement:
            replacement = '(?:'+replacement+')'
        rules[42] = re.sub('(?<![0-9])'+item+'(?![0-9])', replacement, rules[42])
while(re.search("[0-9]", rules[31])):
    for item in num_extract.findall(rules[31]):
        replacement = rules[int(item)]
        if '|' in replacement:
            replacement = '(?:'+replacement+')'
        rules[31] = re.sub('(?<![0-9])'+item+'(?![0-9])', replacement, rules[31])
rules[42] = re.sub(' ', '', rules[42])
rules[42] = re.sub('"', '', rules[42])    
rules[31] = re.sub(' ', '', rules[31])
rules[31] = re.sub('"', '', rules[31])

In [ ]:
# rules[8] = '42 | 42 8', equivalent to 42+
# rules[11] = '42 31 | 42 11 31', equivalent to 42(?R)?31
rule_0 = regex.compile('^(?P<g42>(?:' +rules[42]+ ')+?)(?P<g31>(?:' +rules[31]+ ')+)$' )
matched = [(rule_0.search(d).groupdict()['g42'], rule_0.search(d).groupdict()['g31']) for d in data_values if rule_0.search(d)]
sum([(len(re.findall(rules[42], m[0])) > len(re.findall(rules[31], m[1]))) for m in matched])


# Day 20

This one was fun. Definitely made some mistakes trying to get the tiles to line up.

In [ ]:
data = get_data(20, '\n\n')
# test = get_test_data(19)
data

In [ ]:
data = [d.strip('\n').split('\n') for d in data]

In [ ]:
def parse_tile(tile):
    id = re.findall('[0-9]+', tile[0])[0]
    tile = np.array([list(re.sub('\.', '0', re.sub('#', '1', t))) for t in tile[1:]])
    hashes = []
    # Right, Top, Left Bottom
    hashes.append(int(''.join(tile[0,:]), 2))
    hashes.append(int(''.join(tile[:,0]), 2))
    hashes.append(int(''.join(tile[-1,:]), 2))
    hashes.append(int(''.join(tile[:,-1]), 2))
    hashes.append(int(''.join(tile[0,:])[::-1], 2))
    hashes.append(int(''.join(tile[:,-1])[::-1], 2))
    hashes.append(int(''.join(tile[-1,:])[::-1], 2))
    hashes.append(int(''.join(tile[:,0])[::-1], 2))
    
    return(id, tile, hashes)
    

In [ ]:
class board:
    def __init__(self, data, mode = 'all'):
        self.board = {}     
        self.mode = mode
        self.unplaced = [tile(d) for d in data]
        self.placed = []
        self.frontier = []
#         t = self.unplaced.pop(first_tile)
#         self.place(t, (0,0), first_orientation)
        
    def continue_placement(self):
        location = self.frontier.pop(0)
        for idx, u in enumerate(self.unplaced):
            placed = self.try_to_place(idx, location)
            if placed:
                return True
                
            

    def check_placement(self, tile_idx, location, orientation):
        tile = self.unplaced[tile_idx]
        area = [idx_to_loc(location, i) for i in range(4)]
        hashes = tile.get_hashes((orientation // 4, orientation % 4))        
        for i, a in enumerate(area):
            tile = self.board.get(a)            
            if tile:
                if hashes[i] != tile.get_hashes((tile.flip, tile.rotate))[(i+2)%4]:
                    return False
        return True

    def try_to_place(self, tile_idx, location):
        if not (12 > location[0] >= 0) & (12 > location[1] >= 0):
            return False
        occupied = (new_loc in self.board.keys())
        for orientation in range(8):
            valid = self.check_placement(tile_idx = tile_idx, location=location, orientation = orientation)
            if  valid and not occupied:
                self.place(orientation = orientation, tile_idx = tile_idx, location=location)
                return True
        return False

        
    def place(self, tile_idx, location, orientation):
        tile = self.unplaced.pop(tile_idx)
        self.board[location] = tile
        tile.set_tile((orientation // 4, orientation % 4))
        self.placed.append(tile)
        self.expand_frontier(location)
        area = [idx_to_loc(location, i) for i in range(4)]
#         for a in area:
#             neighbor = self.board.get(a)
#             if neighbor:
#                 print(tile.get_hashes((tile.flip, tile.rotate)), neighbor.get_hashes((neighbor.flip, neighbor.rotate)))
                
        
                      
        
    def expand_frontier(self, location):
        if self.mode == 'all':
            neighborhood = range(4)
        elif self.mode == 'h':
            neighborhood = [1,3]
        elif self.mode == 'v':
            neighborhood = [0,2]            
        for i in neighborhood:
            new_loc = idx_to_loc(location, i)
            if (new_loc not in self.frontier) & (new_loc not in self.board.keys()):
                self.frontier.append(new_loc)
                
    def plot(self):
        points = np.zeros([12,12])
        for item in self.board.keys():
            points[item] = 1
#         x = [p[0] for p in points]
#         y = [p[1] for p in points]
        plt.matshow(points)
                    
        
def idx_to_loc(location, i):
    return(location[0] + (i+1)% 2 * (-2 * ((i+1) // 2)+1), location[1] + (i% 2 * (-2 * (i // 2)+1)))



In [ ]:
class tile:
    def __init__(self, data):
        tile_data = parse_tile(data)
        self.id, self.tile, hashes = tile_data
        self.hashes = np.array([hashes[0:4], hashes[4:8]])
        self.neighbors = 0

    def get_tile(self, flip, rotate):
        tile = self.tile
        tile = np.rot90(tile, rotate)
        if flip:
            tile = np.flip(tile, 0)
        return tile.astype(int)
        
    def get_hashes(self, orientation):
        flip, rotate = orientation
        return(np.append(self.hashes[flip][rotate:], self.hashes[flip][:rotate]))
        
    def set_tile(self, orientation):
        self.flip, self.rotate = orientation
    

In [ ]:
counts = np.bincount(np.array([t.hashes for t in board(data).unplaced]).reshape(-1))
edge_hashes = set(np.where(counts == 1)[0])
np.prod([int(t.id) for t in board(data).unplaced if len(set.intersection(edge_hashes, set(t.hashes.reshape(-1)))) == 4])



In [ ]:
corners = [int(t.id) for t in board(data).unplaced if len(set.intersection(edge_hashes, set(t.hashes.reshape(-1)))) == 4]
corners

### Part 2

In [ ]:
image = board(data)
full_image = np.zeros([12*10,12*10])
tileset = image.unplaced.copy()

plt.imshow(full_image)
last = [idx for idx, u in enumerate(image.unplaced) if u.id == '1543'][0]
tile = tileset[last]
full_image[0:10, 0:10] = tile.get_tile(0, 0)

tileset.remove(tile)

for col in range(1, 12):
    for t_obj in [tileset[idx] for idx, item in enumerate(tileset)]:
        ts = [t_obj.get_tile(flip, rotation) for rotation in range(4) for flip in range(2)]
        for t in ts:
            if np.all(t[:,0] == full_image[0:10,(col*10)-1]):
                full_image[0:10,(col*10):((col*10)+10)] = t
                tileset.remove(t_obj)


    
for col in range(0, 12):
    for row in range(1, 12):
        for t_obj in [tileset[idx] for idx, item in enumerate(tileset) for rotation in range(4) for flip in range(2)]:
            ts = [t_obj.get_tile(flip, rotation) for rotation in range(4) for flip in range(2)]
            for t in ts:
                if np.all(t[0,:] == full_image[(row*10)-1,(col*10):(col*10+10)]):
                    full_image[(row*10):(row*10+10), (col*10):(col*10+10)] = t
                    plt.imshow(full_image)
                    # I don't know why this is getting triggered more than once.
                    try:
                        tileset.remove(t_obj)
                    except:
                        pass


In [ ]:
rowcols_to_drop = np.append(np.arange(0,120, 10), np.arange(9,120, 10))
new_image = np.delete(np.delete(full_image,rowcols_to_drop, axis=1), rowcols_to_drop, axis=0)
plt.imshow(new_image)

In [ ]:
monster = \
'''                  # 
#    ##    ##    ###
 #  #  #  #  #  #   '''

In [ ]:
monster = ([list(l) for l in monster.replace('#', '1').replace(' ', '0').split('\n')])
monster = np.array(monster)
monster_mask = monster.astype('bool')

In [ ]:
monster_cnt = 0

for rot in np.arange(0, 4, 2):
    for row in range(0, 93):
        for col in range(0,76):
            test = new_image[row:(row+3), col:(col+20)]
            if np.all(test[np.rot90(monster_mask,rot)].astype(bool)):
                monster_cnt += 15

for rot in np.arange(1, 4, 2):
    for row in range(0, 76):
        for col in range(0,93):
            test = new_image[row:(row+20), col:(col+3)]
            if np.all(test[np.rot90(monster_mask,rot)].astype(bool)):
                monster_cnt += 15
                
flip_image = np.flip(new_image, axis=1)
for rot in np.arange(0, 4, 2):
    for row in range(0, 93):
        for col in range(0,76):
            test = flip_image[row:(row+3), col:(col+20)]
            if np.all(test[np.rot90(monster_mask,rot)].astype(bool)):
                monster_cnt += 15

for rot in np.arange(1, 4, 2):
    for row in range(0, 76):
        for col in range(0,93):
            test = flip_image[row:(row+20), col:(col+3)]
            if np.all(test[np.rot90(monster_mask,rot)].astype(bool)):
                monster_cnt += 15

In [ ]:
np.sum(new_image) - monster_cnt

# Day 21

In [ ]:
data = get_data(21, '\n')
# test = get_test_data(21)
data

In [ ]:
data[0]

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None) 

In [ ]:
allergens = [list(set(s[0].split(', '))) for s in [re.findall('\(contains (.+)\)', d) for d in data]]
ingredients = [list(set(s[0].split(' '))) for s in [re.findall('(.*)\(contains .+\)', d) for d in data]]

In [ ]:
ingredients = pd.melt(pd.DataFrame(ingredients).reset_index(), id_vars = 'index')
ingredients = ingredients.drop('variable', 1)
ingredients = ingredients[ingredients.value.notnull()]
ingredients = ingredients.rename(columns = {'index':'dish'})
ingredients = ingredients[ingredients.value != '']

In [ ]:
ingredients

In [ ]:
allergens = pd.melt(pd.DataFrame(allergens).reset_index(), id_vars = 'index')
allergens = allergens.drop('variable', 1)
allergens = allergens[allergens.value.notnull()]
allergens = allergens.rename(columns = {'index':'dish'})
allergens = allergens[allergens.value != '']

In [ ]:
allergens

In [ ]:
combined

In [ ]:
combined = pd.merge(ingredients, allergens, on='dish')
combined = combined.rename(columns = {'value_x': 'ingredient', 'value_y': 'allergen'})

In [ ]:
allergen_counts = combined.groupby(['allergen', 'ingredient']).count()

In [ ]:
ingredient_counts = combined.groupby('ingredient').count()[['dish']]

In [ ]:
dish_allergen = ingredient_counts.join(allergen_counts, lsuffix = '_total', rsuffix = '_allergen')

In [ ]:
dish_allergen['proportion'] = dish_allergen['dish_allergen'] / dish_allergen['dish_total']

In [ ]:
dish_allergen

In [ ]:
allergens.value.unique()

In [ ]:
len(ingredients.value.unique())

In [ ]:
# Rules: 
# 1. Each allergen is found in exactly one ingredient.
# 2. Allergens are not always marked
# Therefore: 
# IF food has allergen
# THEN ingredient must be in food
# 
# Process:
# Find all foods with allergens
# Get common ingredients

In [ ]:
allergen_dishes = allergens.groupby('value').apply(lambda x: list(x['dish'])).to_dict()

In [ ]:
likely_allergens = {}
for allergen, dish in allergen_dishes.items():
    ingredients[ingredients.dish.isin(dish)]
    candidates = (ingredients[ingredients.dish.isin(dish)].groupby('value').count() / len(dish)).sort_values('dish', ascending = False) == 1
    likely_allergens[allergen] = set(candidates[candidates.dish].index.to_list())


In [ ]:
likely_allergens

In [ ]:
confirmed_allergens = {}
while len(likely_allergens) > 0:
    k = [k for k, v in likely_allergens.items() if len(v) == 1][0]
    print(k)
    i = likely_allergens.pop(k)
    print(i)
    confirmed_allergens[k] = i
    for k,v in likely_allergens.items():
        likely_allergens[k] = v - i
    
    

In [ ]:
allergens_to_remove = [l.pop() for l in confirmed_allergens.values()]

In [ ]:
ingredients[~ingredients.value.isin(allergens_to_remove)].shape

In [ ]:
# Part 2
allergens_to_remove.sort()

In [ ]:
','.join(allergens_to_remove)

In [ ]:
','.join([confirmed_allergens[k].pop() for k in list(np.sort(list(confirmed_allergens.keys())))])

In [ ]:
confirmed_allergens

In [ ]:
list(np.sort(list(confirmed_allergens.keys())))